In [4]:
import importlib
import numpy as np
import tmd
import os
import sklearn.tree
import sklearn 

import classification_functions as fcts
from typing import List, Dict

In [5]:
data_path = "./Data/Reconstructed/"
folders_to_treat = ["L5_UPC","L5_TPC_A","L5_TPC_B","L5_TPC_C"]
training_neurons = fcts.load_neurons_from_folders(data_path, folders_to_treat)

Neurons loaded from L5_UPC: 27
Neurons loaded from L5_TPC_A: 64
Neurons loaded from L5_TPC_B: 38
Neurons loaded from L5_TPC_C: 30


In [6]:
def load_data(
    all_neurons: Dict[str, List],
    types: List[str],
    neurite_type: str,
    pers_hom_function: str = "radial_distances",
):
    """
    Load persistence diagrams and labels from a neuron dictionary.

    Parameters:
        all_neurons (dict): Dictionary of {group_name: list_of_neurons}
        types (list): List of group names to use
        neurite_type (str): 'basal_dendrite', 'apical_dendrite', 'axon', 'dendrite'
        pers_hom_function (str): Feature type for persistence (e.g. 'radial_distances', 'path')

    Returns:
        labels (list): List of integer labels for each neuron
        pers_diagrams (list): List of persistence diagrams
    """

    labels = []
    pers_diagrams = []

    for label_index, group in enumerate(types):
        neurons = all_neurons[group]
        for neuron in neurons:
            diag = tmd.methods.get_ph_neuron(neuron, feature=pers_hom_function, neurite_type=neurite_type)
            if diag:  # Only keep non-empty diagrams
                pers_diagrams.append(diag)
                labels.append(label_index + 1)  # Label is 1-based index

    return labels, pers_diagrams

In [7]:
def split_data(labels, pers_images, train_size=0.8):
    """
    Split the data into training and testing sets.
    """
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(pers_images, labels, train_size=train_size)
    return X_train, X_test, y_train, y_test

### Data Loading

In [13]:
labels, pers_diagrams = load_data(
    all_neurons=training_neurons,
    types=folders_to_treat,
    neurite_type="apical_dendrite",
    pers_hom_function="radial_distances"
)


### Make images

In [15]:
xlim, ylim = tmd.vectorizations.get_limits(pers_diagrams)
pers_imgs = [tmd.vectorizations.persistence_image_data(pd, xlim=xlim, ylim=ylim).flatten() for pd in pers_diagrams] ## alt use gaussian_image


X_train, X_test, y_train, y_test = split_data(
    labels=labels,
    pers_images=pers_imgs,
    train_size=0.8
)

### Training Dataset

In [16]:
cls = sklearn.tree.DecisionTreeClassifier()
cls.fit(X_train, y_train)

DecisionTreeClassifier()

In [17]:
print(cls.predict(X_test))
print(y_test)
print(cls.score(X_test, y_test))

[1 3 3 4 2 2 2 2 2 4 2 1 2 3 2 3 1 1 2 2 1 1 4 2 2 2 4 4 2 2 2 3]
[1, 3, 3, 1, 2, 2, 4, 3, 4, 4, 3, 1, 4, 3, 3, 3, 1, 4, 2, 3, 1, 1, 1, 2, 2, 4, 1, 3, 2, 2, 2, 3]
0.59375
